In [3]:
library(data.table)
library(spacexr)
library(readr)
library(ggplot2)
library(Seurat)
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ stringr   1.5.1
✔ forcats   1.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ tidyr::expand()      masks Matrix::expand()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ tidyr::pack()        masks Matrix::pack()
✖ lubridate::quarter()

In [2]:
bin_name = 'bin20'
name = 'Y01804JC_9'
group = 'annotation'
spatial = paste0('/data/work/Y01804JC_9/Y01804JC_9_10w.rds') 

output = paste0('/data/work/HSJ/1x1/RCTD/01.script')

if (!dir.exists(output)) dir.create(output, recursive = TRUE); setwd(output)

In [ ]:
colorlist = c("#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941",
             "#006FA6", "#A30059", "#FFE4E1", "#0000A6", "#63FFAC",
             "#B79762", "#004D43", "#8FB0FF", "#997D87", "#5A0007",
             "#809693", "#1B4400", "#4FC601", "#3B5DFF", "#FF2F80",
             "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9",
             "#B903AA", "#DDEFFF", "#7B4F4B", "#A1C299", "#0AA6D8",
             "#00A087FF", "#4DBBD5FF", "#E64B35FF", "#3C5488FF", "#F38400",
             "#A1CAF1", "#C2B280", "#848482", "#E68FAC", "#0067A5",
             "#F99379", "#604E97", "#F6A600", "#B3446C", "#DCD300",
             "#882D17", "#8DB600", "#654522", "#E25822", "#2B3D26",
             "#191970", "#000080","#6495ED", "#1E90FF", "#00BFFF", 
             "#00FFFF", "#FF1493","#FF00FF", "#A020F0", "#63B8FF", 
             "#008B8B", "#54FF9F","#00FF00", "#76EE00", "#FFF68F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F")

## 导入空间组数据
SeuObj = readRDS(spatial)
print("SeuObj read successfully")

## 导入单细胞数据
sc = readRDS(paste0("/data/work/HSJ/1x1/S1_B03404C4.rds")) 
# meta = read.csv(metadata,row.names=1)
# sc@meta.data = meta[colnames(sc),]
# sc = subset(sc,sample == bin_name)

group = group
cell_low <- table(sc@meta.data[[group]])[table(sc@meta.data[[group]]) <25] %>% names()
Idents(sc) <- sc@meta.data[[group]]
sc <- subset(sc ,idents = cell_low, invert = TRUE)
sc@meta.data[[group]] <- as.factor(sc@meta.data[[group]])
sc=subset(sc, downsample = 500) # sc@meta.data[[group]] 列 downsampl 提提速

print(table(sc@meta.data[[group]]))
print("sc read successfully！")



In [5]:


###提取空间组数据的信息

# 直接赋值，保持 dgCMatrix 稀疏格式
exp_spatial = SeuObj@assays$Spatial@counts

# exp_spatial=as.matrix(SeuObj@assays$Spatial@counts)
# exp_spatial=as.data.frame(exp_spatial)  # RCTD 支持 matrix/dgCMatrix，不需要 dataframe

SeuObj$coor_y = SeuObj@images[[names(SeuObj@images)]]@coordinates$row
SeuObj$coor_x = SeuObj@images[[names(SeuObj@images)]]@coordinates$col
coord <- data.frame(stereo_1 = SeuObj$coor_x, stereo_2 = SeuObj$coor_y)
SeuObj[["stereo"]] <- SeuratObject::CreateDimReducObject(embeddings = as.matrix(coord), key = "stereo_", assay = "Spatial")

row <- SeuObj$coor_y
col <- SeuObj$coor_x
coord_spatial <- data.frame(row,col)
rownames(coord_spatial)<- Cells(SeuObj)
nUMI_spatial=SeuObj@meta.data[,"nCount_Spatial"]
names(nUMI_spatial)=rownames(SeuObj@meta.data)

###提取单细胞的数据
# 直接赋值，保持 dgCMatrix 稀疏格式
exp_sc = sc@assays$RNA@counts
# exp_sc=as.matrix(sc@assays$RNA@counts)
# exp_sc=as.data.frame(exp_sc) # RCTD 支持 matrix/dgCMatrix，不需要 dataframe

celltype_sc=sc@meta.data[[group]]
celltype_sc <- gsub("[/\\\\ ]", "_", celltype_sc)  # 替换 / \ 和空格
celltype_sc <- as.factor(celltype_sc)
names(celltype_sc)=rownames(sc@meta.data)
celltype_sc=as.factor(celltype_sc)
nUMI_sc=sc@meta.data[,"nCount_RNA"]
names(nUMI_sc)=rownames(sc@meta.data)

###构建RCTD的数据数据（puck和reference）
reference <- Reference(exp_sc, celltype_sc, nUMI_sc)
puck <- SpatialRNA(coord_spatial, exp_spatial, nUMI_spatial)
print("data have praperad successfully！")

###清除内存
rm(sc)
gc()


[1] "data have praperad successfully！"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3417259,182.6,6449302,344.5,5015937,267.9
Vcells,291641655,2225.1,10478917343,79947.8,13098644514,99934.8


In [ ]:
print('\n ------------------- 开始 run.RCTD ！-------------------- \n')
myRCTD <- create.RCTD(puck, reference, max_cores = 10, UMI_min=0, UMI_min_sigma = 100)
myRCTD <- run.RCTD(myRCTD, doublet_mode = 'full')

saveRDS(myRCTD, file = paste0(bin_name, "_myRCTD_full.rds"))
print("RCTD successfully！")

results_df=myRCTD@results[["weights"]]
a=as.data.frame(myRCTD@results$weights)
write.csv(a,paste0("RCTD_full_weights.csv"),quote = FALSE)

[1] "\n ------------------- 开始 run.RCTD ！-------------------- \n"


Begin: process_cell_type_info

process_cell_type_info: number of cells in reference: 5163

process_cell_type_info: number of genes in reference: 53030




         CD14_CD16_monocyte               CD14_monocyte 
                        418                         500 
              CD16_monocyte                CD16_NK_cell 
                         43                         495 
      Cytotoxic_CD8+_T_cell              Dendritic_cell 
                        500                          50 
Effector_memory_CD8+_T_cell               Megakaryocyte 
                        364                          26 
              Memory_B_cell          Memory_CD4+_T_cell 
                        153                         500 
               Naive_B_cell            Naive_CD4_T_cell 
                        500                         500 
         Naive_CD8+_T_cells                    NKT_cell 
                        500                         450 
          Regulatory_T_cell 
                        164 


End: process_cell_type_info

create.RCTD: getting regression differentially expressed genes: 

get_de_genes: CD14_CD16_monocyte found DE genes: 237

get_de_genes: CD14_monocyte found DE genes: 256

get_de_genes: CD16_monocyte found DE genes: 211

get_de_genes: CD16_NK_cell found DE genes: 98

get_de_genes: Cytotoxic_CD8+_T_cell found DE genes: 50

get_de_genes: Dendritic_cell found DE genes: 200

get_de_genes: Effector_memory_CD8+_T_cell found DE genes: 70

get_de_genes: Megakaryocyte found DE genes: 594

get_de_genes: Memory_B_cell found DE genes: 126

get_de_genes: Memory_CD4+_T_cell found DE genes: 30

get_de_genes: Naive_B_cell found DE genes: 165

get_de_genes: Naive_CD4_T_cell found DE genes: 76

get_de_genes: Naive_CD8+_T_cells found DE genes: 103

get_de_genes: NKT_cell found DE genes: 34

get_de_genes: Regulatory_T_cell found DE genes: 63

get_de_genes: total DE genes: 1498

create.RCTD: getting platform effect normalization differentially expressed genes: 

get_de_genes: CD14

In [ ]:
# myRCTD <- readRDS('./bin20_myRCTD_full.rds')
# str(myRCTD)

In [11]:
colnames(myRCTD@results[["weights"]])

[1] "CD14_CD16_monocyte"          "CD14_monocyte"              
 [3] "CD16_monocyte"               "CD16_NK_cell"               
 [5] "Cytotoxic_CD8+_T_cell"       "Dendritic_cell"             
 [7] "Effector_memory_CD8+_T_cell" "Megakaryocyte"              
 [9] "Memory_B_cell"               "Memory_CD4+_T_cell"         
[11] "Naive_B_cell"                "Naive_CD4_T_cell"           
[13] "Naive_CD8+_T_cells"          "NKT_cell"                   
[15] "Regulatory_T_cell"

In [10]:
head(myRCTD@results[["weights"]])

  [[ suppressing 15 column names ‘CD14_CD16_monocyte’, ‘CD14_monocyte’, ‘CD16_monocyte’ ... ]]



6 x 15 sparse Matrix of class "dgCMatrix"
                                                                          
BIN.1247994 0.0001085609 0.247965439 0.112848805 0.0001085609 0.0001085609
BIN.1119621 0.1227551997 0.000155087 0.040459605 0.0001550870 0.0495251065
BIN.409017  0.0001085609 0.122530607 0.002792011 0.0001085609 0.0001085609
BIN.1093508 0.0001550870 0.157298707 0.112933399 0.0497477389 0.0416107059
BIN.270637  0.0001085609 0.065765489 0.128614876 0.0084075978 0.0907441315
BIN.801900  0.0006464711 0.151799769 0.164686657 0.0001085609 0.0001085609
                                                                            
BIN.1247994 0.0001085609 0.0001085609 0.0001085609 0.0143246700 0.0235989282
BIN.1119621 0.0001550870 0.0001550870 0.0194765218 0.1166919627 0.0001550870
BIN.409017  0.0001085609 0.0778840303 0.0118570288 0.0001085609 0.1157026065
BIN.1093508 0.0674511118 0.0001550870 0.0075935817 0.0001550870 0.1843799336
BIN.270637  0.0001085609 0.0001085609 0.00010856

In [12]:
library(ggplot2)
library(dplyr)
library(tibble)

# 1. 提取权重矩阵 (dgCMatrix -> matrix -> data.frame)
weights_matrix <- as.matrix(myRCTD@results$weights)

# 2. 归一化权重 (让每个 Spot 的所有细胞类型权重之和为 1)
# 这一步对于 Full Mode 很重要，因为原始权重可能不严格等于 1
norm_weights <- sweep(weights_matrix, 1, rowSums(weights_matrix), '/')
norm_weights[is.na(norm_weights)] <- 0 # 防止除以0产生的NA

# 3. 提取坐标
coords <- myRCTD@spatialRNA@coords
colnames(coords) <- c("x", "y")

# 4. 合并数据
plot_data <- cbind(coords, norm_weights)
head(plot_data)

,x,y,CD14_CD16_monocyte,CD14_monocyte,CD16_monocyte,CD16_NK_cell,Cytotoxic_CD8+_T_cell,Dendritic_cell,Effector_memory_CD8+_T_cell,Megakaryocyte,Memory_B_cell,Memory_CD4+_T_cell,Naive_B_cell,Naive_CD4_T_cell,Naive_CD8+_T_cells,NKT_cell,Regulatory_T_cell
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BIN.1247994,1062,258,0.0001436266,0.3280594577,0.149299506,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0189516066,0.0312214944,0.0001436266,0.0058319787,0.2891848354,0.1763021080,0.0001436266
BIN.1119621,953,69,0.1739653475,0.0002197851,0.057338257,0.0002197851,0.0701856409,0.0002197851,0.0002197851,0.0276015997,0.1653726921,0.0002197851,0.0002197851,0.0002197851,0.2728863493,0.0578715224,0.1732400951
BIN.409017,348,945,0.0001625905,0.1835127599,0.004181564,0.0001625905,0.0001625905,0.0001625905,0.1166460665,0.0177581433,0.0001625905,0.1732865375,0.0001625905,0.1911296690,0.3121845356,0.0001625905,0.0001625905
BIN.1093508,930,1004,0.0002492335,0.2527878219,0.181490290,0.0799473994,0.0668707322,0.1083977100,0.0002492335,0.0122033105,0.0002492335,0.2963088678,0.0002492335,0.0002492335,0.0002492335,0.0002492335,0.0002492335
BIN.270637,231,157,0.0001745293,0.1057286835,0.206769261,0.0135165763,0.1458859009,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.1322499128,0.3942789020,0.0001745293,0.0001745293,0.0001745293
BIN.801900,682,1044,0.0009672768,0.2271291125,0.246411009,0.0001624333,0.0001624333,0.1239902331,0.0001624333,0.0064621144,0.0001624333,0.3613584437,0.0001624333,0.0323823438,0.0001624333,0.0001624333,0.0001624333


In [13]:
# 计算每个 Spot 权重最大的细胞类型
major_cell_type <- colnames(norm_weights)[max.col(norm_weights, ties.method = "first")]
plot_data$major_type <- major_cell_type
head(plot_data)

,x,y,CD14_CD16_monocyte,CD14_monocyte,CD16_monocyte,CD16_NK_cell,Cytotoxic_CD8+_T_cell,Dendritic_cell,Effector_memory_CD8+_T_cell,Megakaryocyte,Memory_B_cell,Memory_CD4+_T_cell,Naive_B_cell,Naive_CD4_T_cell,Naive_CD8+_T_cells,NKT_cell,Regulatory_T_cell,major_type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
BIN.1247994,1062,258,0.0001436266,0.3280594577,0.149299506,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0189516066,0.0312214944,0.0001436266,0.0058319787,0.2891848354,0.1763021080,0.0001436266,CD14_monocyte
BIN.1119621,953,69,0.1739653475,0.0002197851,0.057338257,0.0002197851,0.0701856409,0.0002197851,0.0002197851,0.0276015997,0.1653726921,0.0002197851,0.0002197851,0.0002197851,0.2728863493,0.0578715224,0.1732400951,Naive_CD8+_T_cells
BIN.409017,348,945,0.0001625905,0.1835127599,0.004181564,0.0001625905,0.0001625905,0.0001625905,0.1166460665,0.0177581433,0.0001625905,0.1732865375,0.0001625905,0.1911296690,0.3121845356,0.0001625905,0.0001625905,Naive_CD8+_T_cells
BIN.1093508,930,1004,0.0002492335,0.2527878219,0.181490290,0.0799473994,0.0668707322,0.1083977100,0.0002492335,0.0122033105,0.0002492335,0.2963088678,0.0002492335,0.0002492335,0.0002492335,0.0002492335,0.0002492335,Memory_CD4+_T_cell
BIN.270637,231,157,0.0001745293,0.1057286835,0.206769261,0.0135165763,0.1458859009,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.1322499128,0.3942789020,0.0001745293,0.0001745293,0.0001745293,Naive_CD4_T_cell
BIN.801900,682,1044,0.0009672768,0.2271291125,0.246411009,0.0001624333,0.0001624333,0.1239902331,0.0001624333,0.0064621144,0.0001624333,0.3613584437,0.0001624333,0.0323823438,0.0001624333,0.0001624333,0.0001624333,Memory_CD4+_T_cell


In [19]:
# --- 开始绘图 (模拟你之前的 Cell Type 分布图) ---

# 定义你的颜色 (保持你原代码的 colorlist)
cell_name <- sort(unique(plot_data$major_type)) # 确保颜色对应
# 注意：如果细胞类型数量超过了 colorlist 长度，需要扩展颜色
use_colors <- colorlist[1:length(cell_name)]
names(use_colors) <- cell_name

p1 <- ggplot(plot_data, aes(x=x, y=y, fill=major_type)) +
  geom_tile() + 
  coord_fixed() +
  labs(title = "Dominant Cell Type (Full Mode Inference)") +
  theme_void() +
  theme(
    panel.background = element_rect(fill = "black"),
    plot.background = element_rect(fill = "black"),
    text = element_text(color = "white"),
    plot.title = element_text(color = "white", hjust = 0.5),
    legend.position = "right",
    legend.text = element_text(color = "white")
  ) +
  scale_fill_manual(values = use_colors)

# 保存
ggsave(paste0(bin_name, "_fullmode_dominant_celltype.pdf"), plot=p1, height = 10, width = 12)


write.csv(plot_data,paste0("RCTD_full_plot_data.csv"),quote = FALSE)


# # 将推断的类型存回 Seurat 对象 (如果你需要的话)
# SeuObj <- AddMetaData(SeuObj, metadata = plot_data$major_type, col.name = "RCTD_full_major")

In [22]:
setwd('/data/work/HSJ/1x1/RCTD/02.result/full')

# 定义绘图函数
plot_spatial_weight <- function(data, cell_type, title) {
  ggplot(data, aes(x=x, y=y, fill=.data[[cell_type]])) +
    geom_tile() +
    coord_fixed() +
    scale_fill_gradientn(colours = c("#3E4A89FF", "#B4DE2CFF", "#FF0000"), limits = c(0,1)) + 
    labs(title = title) +
    theme_void() +
    theme(
      panel.background = element_rect(fill = "black"),
      plot.background = element_rect(fill = "black"),
      text = element_text(color = "white"),
      plot.title = element_text(color = "white", hjust = 0.5),
      legend.text = element_text(color = "white")
    )
}

# 批量绘图
pdf(paste0(bin_name, "_fullmode_weights.pdf"), height=10, width=10, bg = "black")

all_cell_types <- colnames(norm_weights)
for (ct in all_cell_types) {
  print(plot_spatial_weight(plot_data, ct, ct))
}

dev.off()

pdf 
  2